Перед запуском убедитесь, что в корне проекта есть файл .env и в нем заполнены выданные вам креды подключения к базам данных и хранилищу

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine

In [3]:
# подгружаем .env
load_dotenv()

True

In [4]:
# Считываем все креды
src_host = os.environ.get('DB_SOURCE_HOST')
src_port = os.environ.get('DB_SOURCE_PORT')
src_username = os.environ.get('DB_SOURCE_USER')
src_password = os.environ.get('DB_SOURCE_PASSWORD')
src_db = os.environ.get('DB_SOURCE_NAME') 

dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

s3_bucket = os.environ.get('S3_BUCKET_NAME')
s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
s3_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

In [5]:
# Создадим соединения
src_conn = create_engine(f'postgresql://{src_username}:{src_password}@{src_host}:{src_port}/{src_db}')
dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')

In [6]:
# Таблица квартиры
TABLE = 'flats_churn'
SQL = f'select * from {TABLE}'
data = pd.read_sql(SQL, dst_conn)
data

,id,flat_id,price,target,floor,kitchen_area,living_area,rooms,is_apartment,studio,total_area,build_year,build_age,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator
0,218515,77152,12200000.0,12200000.0,9,10.0,46.000000,3,false,false,57.0,1968,57,1,55.799854,37.743679,2.64,307,9,true
1,218519,77156,7350000.0,7350000.0,5,6.2,29.000000,2,false,false,45.0,1960,65,1,55.824314,37.765388,2.48,60,5,false
2,77158,77157,8000000.0,8000000.0,14,7.4,19.100000,1,false,false,38.0,2003,22,4,55.805370,37.814114,2.70,245,17,true
3,218521,77158,5500000.0,5500000.0,15,11.0,16.500000,1,false,false,40.0,2020,5,4,55.965698,37.150665,2.66,619,15,true
4,218522,77159,7550000.0,7550000.0,8,6.5,33.700001,3,false,false,51.0,1969,56,4,55.702065,37.814346,2.50,324,9,true
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81609,218510,77147,7500000.0,7500000.0,5,7.0,18.799999,1,false,false,38.0,2004,21,4,55.808247,37.816605,2.64,378,14,true
81610,77149,77148,8300000.0,8300000.0,8,6.4,23.500000,2,false,false,40.0,1968,57,6,55.801041,37.726288,2.64,84,12,true
81611,218512,77149,6500000.0,6500000.0,12,8.2,17.000000,1,false,false,41.0,1978,47,4,55.852455,37.617645,2.50,191,12,true
81612,77151,77150,6200000.0,6200000.0,14,7.5,20.000000,1,false,false,38.0,2001,24,4,55.659412,37.772747,2.74,179,15,true


In [7]:
data.drop(columns=['studio', 'price', 'build_year'], inplace=True) #т.к только false значения
data

,id,flat_id,target,floor,kitchen_area,living_area,rooms,is_apartment,total_area,build_age,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator
0,218515,77152,12200000.0,9,10.0,46.000000,3,false,57.0,57,1,55.799854,37.743679,2.64,307,9,true
1,218519,77156,7350000.0,5,6.2,29.000000,2,false,45.0,65,1,55.824314,37.765388,2.48,60,5,false
2,77158,77157,8000000.0,14,7.4,19.100000,1,false,38.0,22,4,55.805370,37.814114,2.70,245,17,true
3,218521,77158,5500000.0,15,11.0,16.500000,1,false,40.0,5,4,55.965698,37.150665,2.66,619,15,true
4,218522,77159,7550000.0,8,6.5,33.700001,3,false,51.0,56,4,55.702065,37.814346,2.50,324,9,true
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81609,218510,77147,7500000.0,5,7.0,18.799999,1,false,38.0,21,4,55.808247,37.816605,2.64,378,14,true
81610,77149,77148,8300000.0,8,6.4,23.500000,2,false,40.0,57,6,55.801041,37.726288,2.64,84,12,true
81611,218512,77149,6500000.0,12,8.2,17.000000,1,false,41.0,47,4,55.852455,37.617645,2.50,191,12,true
81612,77151,77150,6200000.0,14,7.5,20.000000,1,false,38.0,24,4,55.659412,37.772747,2.74,179,15,true


In [8]:
## 

# Кодируем бинарные признаки

In [9]:
cat_features = data.select_dtypes(include='object')

In [10]:
potential_binary_features = cat_features.nunique() == 2
potential_binary_features

is_apartment    True
has_elevator    True
dtype: bool

In [11]:
binary_cat_features = cat_features[potential_binary_features[potential_binary_features].index]
binary_cat_features

,is_apartment,has_elevator
0,false,true
1,false,false
2,false,true
3,false,true
4,false,true
...,...,...
81609,false,true
81610,false,true
81611,false,true
81612,false,true


In [12]:
from sklearn.preprocessing import OneHotEncoder

one_hot_drop = OneHotEncoder(drop='if_binary', sparse_output=False) 
drop_res = one_hot_drop.fit_transform(binary_cat_features)
drop_res = pd.DataFrame(drop_res, columns=one_hot_drop.get_feature_names_out())
# print(drop_res.head())
# print('shape: ', drop_res.shape) 
drop_res

,is_apartment_true,has_elevator_true
0,0.0,1.0
1,0.0,0.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0
...,...,...
81609,0.0,1.0
81610,0.0,1.0
81611,0.0,1.0
81612,0.0,1.0


# Нормализация

## Нормализуем категориальные INT колонки

In [13]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
Min_Max_Columns = scaler.fit_transform(data[['build_age', 'floor', 'rooms', 'floors_total']])
Min_Max_Columns

array([[0.4954955 , 0.42105263, 0.5       , 0.28571429],
       [0.56756757, 0.21052632, 0.25      , 0.14285714],
       [0.18018018, 0.68421053, 0.        , 0.57142857],
       ...,
       [0.40540541, 0.57894737, 0.        , 0.39285714],
       [0.1981982 , 0.68421053, 0.        , 0.5       ],
       [0.51351351, 0.21052632, 0.        , 0.28571429]])

## Нормализуем int onehotencoder

In [14]:
data['building_type_int'].unique()

array([1, 4, 6, 2, 3, 0])

In [15]:
from sklearn.preprocessing import OneHotEncoder

one_hot_drop = OneHotEncoder(sparse_output=False)
building_type = one_hot_drop.fit_transform(data[['building_type_int']])
print('shape: ', building_type.shape)
building_type

shape:  (81614, 6)


array([[0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       ...,
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.]])

# Нормируем числовые признаки float

In [16]:
num_features = data.select_dtypes(['float'])
num_features.drop(columns=['target',], inplace=True)
#num_features.drop(columns=['target',], inplace=True) # Попробовать закодировать координатые отдельно
num_features

,kitchen_area,living_area,total_area,latitude,longitude,ceiling_height
0,10.0,46.000000,57.0,55.799854,37.743679,2.64
1,6.2,29.000000,45.0,55.824314,37.765388,2.48
2,7.4,19.100000,38.0,55.805370,37.814114,2.70
3,11.0,16.500000,40.0,55.965698,37.150665,2.66
4,6.5,33.700001,51.0,55.702065,37.814346,2.50
...,...,...,...,...,...,...
81609,7.0,18.799999,38.0,55.808247,37.816605,2.64
81610,6.4,23.500000,40.0,55.801041,37.726288,2.64
81611,8.2,17.000000,41.0,55.852455,37.617645,2.50
81612,7.5,20.000000,38.0,55.659412,37.772747,2.74


In [17]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler_res = scaler.fit_transform(num_features)
print(scaler_res)
print(pd.DataFrame(scaler_res, columns=scaler.get_feature_names_out()))

[[ 0.73740727  1.53347382  0.42995329  0.64802132  0.90533611 -0.32038391]
 [-1.10394136 -0.11623823 -0.3572636   0.87394072  1.04199161 -1.60916195]
 [-0.52246274 -1.07695286 -0.81647344  0.69896945  1.34870409  0.16290714]
 ...
 [-0.13481055 -1.28074086 -0.61966922  1.13386079  0.11198502 -1.44806494]
 [-0.47400624 -0.9896152  -0.81647344 -0.64914789  1.08831185  0.48510117]
 [-0.71628895 -1.08665709 -1.17728118  0.61934097  0.78097505 -0.32038391]]
       kitchen_area  living_area  total_area  latitude  longitude  \
0          0.737407     1.533474    0.429953  0.648021   0.905336   
1         -1.103941    -0.116238   -0.357264  0.873941   1.041992   
2         -0.522463    -1.076953   -0.816473  0.698969   1.348704   
3          1.221973    -1.329262   -0.685271  2.179813  -2.827537   
4         -0.958572     0.339859    0.036345 -0.255198   1.350169   
...             ...          ...         ...       ...        ...   
81609     -0.716289    -1.106066   -0.816473  0.725536   1.36

# Результат

In [18]:
import pandas as pd
import yaml
import os
import joblib
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from catboost import CatBoostClassifier
from category_encoders import CatBoostEncoder
from catboost import CatBoostRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from category_encoders import CatBoostEncoder

In [19]:
cat_features = data.select_dtypes(include='object')
cat_features

,is_apartment,has_elevator
0,false,true
1,false,false
2,false,true
3,false,true
4,false,true
...,...,...
81609,false,true
81610,false,true
81611,false,true
81612,false,true


In [20]:
# onehotencoder
cat_features = data.select_dtypes(include='object')
potential_binary_features = cat_features.nunique() == 2
binary_cat_features = cat_features[potential_binary_features[potential_binary_features].index]
binary_cat_features

,is_apartment,has_elevator
0,false,true
1,false,false
2,false,true
3,false,true
4,false,true
...,...,...
81609,false,true
81610,false,true
81611,false,true
81612,false,true


In [21]:
OneHotEncoder_Columns = data[['building_type_int']]
OneHotEncoder_Columns

,building_type_int
0,1
1,1
2,4
3,4
4,4
...,...
81609,4
81610,6
81611,4
81612,4


In [22]:
# MinMaxScaler
Min_Max_Columns = data[['build_age', 'floor', 'rooms', 'floors_total']]
Min_Max_Columns

,build_age,floor,rooms,floors_total
0,57,9,3,9
1,65,5,2,5
2,22,14,1,17
3,5,15,1,15
4,56,8,3,9
...,...,...,...,...
81609,21,5,1,14
81610,57,8,2,12
81611,47,12,1,12
81612,24,14,1,15


In [23]:
# StandardScaler
num_features = data.select_dtypes(['float'])
num_features.drop(columns=['target',], inplace=True)
num_features

,kitchen_area,living_area,total_area,latitude,longitude,ceiling_height
0,10.0,46.000000,57.0,55.799854,37.743679,2.64
1,6.2,29.000000,45.0,55.824314,37.765388,2.48
2,7.4,19.100000,38.0,55.805370,37.814114,2.70
3,11.0,16.500000,40.0,55.965698,37.150665,2.66
4,6.5,33.700001,51.0,55.702065,37.814346,2.50
...,...,...,...,...,...,...
81609,7.0,18.799999,38.0,55.808247,37.816605,2.64
81610,6.4,23.500000,40.0,55.801041,37.726288,2.64
81611,8.2,17.000000,41.0,55.852455,37.617645,2.50
81612,7.5,20.000000,38.0,55.659412,37.772747,2.74


In [24]:
one_hot_encoder_features = pd.concat([binary_cat_features, data[['building_type_int']]], axis=1)
one_hot_encoder_features

,is_apartment,has_elevator,building_type_int
0,false,true,1
1,false,false,1
2,false,true,4
3,false,true,4
4,false,true,4
...,...,...,...
81609,false,true,4
81610,false,true,6
81611,false,true,4
81612,false,true,4


## Обучим модель

In [25]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(
    data, 
    data['target'], 
    test_size=0.2,   # 20% данных валидация, 80% обучение
    random_state=42, # Фиксируем случайность для воспроизводимости
    shuffle=True      # Перемешиваем данные перед разбиением
)

In [38]:
preprocessor = ColumnTransformer(
    [
    ('binary', OneHotEncoder(drop='if_binary'), binary_cat_features.columns.tolist()),
    ('one_hot', OneHotEncoder(), OneHotEncoder_Columns.columns.tolist()),
    ('minmax', MinMaxScaler(), Min_Max_Columns.columns.tolist()),
    ('num', 'passthrough', num_features.columns.tolist())
    ],
    remainder='drop',
    verbose_feature_names_out=False
)

In [39]:
# Визуализируем
data_transformed = preprocessor.fit_transform(data, data['target'])
print(data_transformed)
print(pd.DataFrame(data_transformed, columns=preprocessor.get_feature_names_out()))

[[ 0.          1.          0.         ... 55.79985428 37.74367905
   2.6400001 ]
 [ 0.          0.          0.         ... 55.82431412 37.76538849
   2.48000002]
 [ 0.          1.          0.         ... 55.80537033 37.81411362
   2.70000005]
 ...
 [ 0.          1.          0.         ... 55.85245514 37.61764526
   2.5       ]
 [ 0.          1.          0.         ... 55.65941238 37.77274704
   2.74000001]
 [ 0.          1.          0.         ... 55.79674911 37.72392273
   2.6400001 ]]
       is_apartment_true  has_elevator_true  building_type_int_0  \
0                    0.0                1.0                  0.0   
1                    0.0                0.0                  0.0   
2                    0.0                1.0                  0.0   
3                    0.0                1.0                  0.0   
4                    0.0                1.0                  0.0   
...                  ...                ...                  ...   
81609                0.0        

In [40]:
model = CatBoostRegressor(
    iterations=1000, 
    depth=8, 
    learning_rate=0.05, 
    l2_leaf_reg=3, 
    random_seed=42,
    verbose=100)

In [41]:
# pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)]
)

In [42]:
# Обучаем pipeline
pipeline.fit(X_tr, y_tr)

0:	learn: 3629916.4722149	total: 17.6ms	remaining: 17.6s
100:	learn: 2202715.6746428	total: 1.65s	remaining: 14.7s
200:	learn: 2137139.8650674	total: 3.15s	remaining: 12.5s
300:	learn: 2098427.1220248	total: 5.17s	remaining: 12s
400:	learn: 2067207.3732285	total: 8.49s	remaining: 12.7s
500:	learn: 2040349.5579246	total: 10.1s	remaining: 10.1s
600:	learn: 2016505.2984977	total: 11.6s	remaining: 7.68s
700:	learn: 1995073.3717902	total: 13.1s	remaining: 5.57s
800:	learn: 1975435.9743059	total: 14.9s	remaining: 3.69s
900:	learn: 1957088.6546414	total: 16.2s	remaining: 1.78s
999:	learn: 1940633.3645855	total: 18.2s	remaining: 0us


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('binary',
                                                  OneHotEncoder(drop='if_binary'),
                                                  ['is_apartment',
                                                   'has_elevator']),
                                                 ('one_hot', OneHotEncoder(),
                                                  ['building_type_int']),
                                                 ('minmax', MinMaxScaler(),
                                                  ['build_age', 'floor',
                                                   'rooms', 'floors_total']),
                                                 ('num', 'passthrough',
                                                  ['kitchen_area',
                                                   'living_area', 'total_area',
                                                   'latitude', 'longitude',
                                                   'ceiling_height'])],
                                   verbose_feature_names_out=False)),
                ('model',
                 <catboost.core.CatBoostRegressor object at 0x7fe7add44850>)])

In [43]:
# получаем предсказания для тестовой выборки
y_pred = pipeline.predict(X_val)

In [44]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Выводим метрики
print('MAE:', mean_absolute_error(y_val, y_pred))   # Средняя абсолютная ошибка
print('MSE:', mean_squared_error(y_val, y_pred))   # Среднеквадратичная ошибка
print('RMSE:', mean_squared_error(y_val, y_pred, squared=False))  # Корень из MSE
print('R²:', r2_score(y_val, y_pred))   # Коэффициент детерминации (качество модели)

MAE: 1661386.375030472
MSE: 4325037133348.1816
RMSE: 2079672.3620195999
R²: 0.6836128748649888


/home/mle_projects/mle-project-sprint-1-v001/.venv_project_1/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


## Кроссвалидация

In [48]:
from sklearn.model_selection import KFold, cross_validate
n_splits = 5
n_jobs = -1
target_col = 'target'

# Используем KFold вместо StratifiedKFold, так как задача - регрессия
cv_strategy = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Метрики для регрессии (RMSE, R^2, MAE)
scoring = ['neg_root_mean_squared_error', 'r2', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error']
cv_res = cross_validate(
        pipeline,
        data.drop(columns=[target_col]),
        data[target_col],
        cv=cv_strategy,
        n_jobs=n_jobs,
        scoring=scoring
    )

In [49]:
for key, value in cv_res.items():
    cv_res[key] = round(value.mean(), 3)

In [50]:
cv_res

{'fit_time': 56.229,
 'score_time': 0.306,
 'test_neg_root_mean_squared_error': -2090751.892,
 'test_r2': 0.684,
 'test_neg_mean_absolute_error': -1673030.969,
 'test_neg_mean_absolute_percentage_error': -0.16}

0:	learn: 3629078.2849007	total: 133ms	remaining: 2m 12s
100:	learn: 2201011.2601580	total: 5.81s	remaining: 51.7s
200:	learn: 2137872.3683076	total: 11.8s	remaining: 46.9s
300:	learn: 2098699.0279603	total: 16.7s	remaining: 38.8s
400:	learn: 2067290.4137206	total: 22.3s	remaining: 33.2s
500:	learn: 2038856.4068004	total: 32.2s	remaining: 32.1s
600:	learn: 2015426.4706749	total: 38.2s	remaining: 25.3s
700:	learn: 1994010.4340535	total: 43.7s	remaining: 18.6s
800:	learn: 1973677.3189051	total: 48.9s	remaining: 12.1s
900:	learn: 1955181.3344818	total: 54.1s	remaining: 5.94s
999:	learn: 1937561.6575458	total: 57.7s	remaining: 0us
0:	learn: 3611817.8785390	total: 18.4ms	remaining: 18.4s
100:	learn: 2193639.8913937	total: 5.43s	remaining: 48.3s
200:	learn: 2127292.8047787	total: 13.5s	remaining: 53.6s
300:	learn: 2087621.6220178	total: 17.6s	remaining: 40.8s
400:	learn: 2055858.2746953	total: 22.1s	remaining: 33s
500:	learn: 2029438.3718478	total: 27.4s	remaining: 27.3s
600:	learn: 2005896.

In [51]:
avg_results = {metric: round(cv_res[f'test_{metric}'].mean(), 3) for metric in scoring}

print("Cross-validation results:", avg_results)

Cross-validation results: {'neg_root_mean_squared_error': -2099148.204, 'r2': 0.672, 'neg_mean_absolute_error': -1680116.407}
0:	learn: 3577170.1926987	total: 93.4ms	remaining: 1m 33s
100:	learn: 2189946.1277893	total: 3.26s	remaining: 29s
200:	learn: 2122440.5754587	total: 6.37s	remaining: 25.3s
300:	learn: 2082458.7427610	total: 9.52s	remaining: 22.1s
400:	learn: 2051766.0962514	total: 12.6s	remaining: 18.8s
500:	learn: 2025445.1238127	total: 15.7s	remaining: 15.6s
600:	learn: 2000467.6653291	total: 18.9s	remaining: 12.6s
700:	learn: 1978101.0433408	total: 22.1s	remaining: 9.44s
800:	learn: 1958108.8445348	total: 25.3s	remaining: 6.29s
900:	learn: 1940212.9014052	total: 28.5s	remaining: 3.13s
999:	learn: 1922881.6013862	total: 31.3s	remaining: 0us
0:	learn: 3563868.7350051	total: 23.7ms	remaining: 23.7s
100:	learn: 2199030.0808863	total: 2.89s	remaining: 25.7s
200:	learn: 2134441.3558143	total: 6.03s	remaining: 24s
300:	learn: 2092340.5029213	total: 9.22s	remaining: 21.4s
400:	learn: